In [1]:
import pandas as pd
from gurobipy import *

In [2]:
with open('test_case.txt') as text_file:
    data = text_file.read().splitlines()

data

['Test Sequence with length 20:',
 'CGUCUUCACUACAGCAUCGG',
 '',
 'Test Sequence with length 30:',
 'GACCUUACUGGGUACGAUUUACUGGAGGAC',
 '',
 'Test Sequence with length 50:',
 'GGCCAGACUGGUGGUGUGACUCCAGGCUAACCGGAUACGCGUGCCUCGGG',
 '',
 'Test Sequence with length 100:',
 'UAUGCAGGUCGCGUUUUUCCACUGCCUAGAUAGCUCUGAGGGUACACUUAGUUCAGCACAUAAGAGGGAUCAUACUAGGUCCGCGUCUUACCUCCUACGA',
 '',
 'Test Sequence with length 200:',
 'CCAUGUGGACGAGUUCACGUUGCGGUCUGCCGCGAAGAAUGAGUCGUCCAAUUUUUUUUUCCUGUAUGACGGAUAGAGUGUAGGGUGGGUGAUAGUGUAACUCCGUCCCGGCGUUGUCUAUGGCAGUCUAAUUGUAGUGAGCCAAAAUAUUUCAGAGAGUAACCAUUACAGGCUACCACACGACCCUGAUCCUAUUGUAU',
 '']

In [3]:
s = data[1]
len(s)

20

In [4]:
m = Model("Crossing_10")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-12


In [5]:
m.setParam('OutputFlag',True)
m.setParam('TimeLimit', 5*60)

Set parameter TimeLimit to value 300


# Pre-Processing Data

In [6]:
ONE_N = list(range(1,len(s)+1))
ONE_N

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

## A_i, U_i, C_i, G_i

In [7]:
a = {}
u = {}
c = {}
g = {}

In [8]:
index = 1
for n in s:
    if (n == 'A'):
        a[index] = 1
        u[index] = 0
        c[index] = 0
        g[index] = 0
    if (n == 'U'):
        a[index] = 0
        u[index] = 1
        c[index] = 0
        g[index] = 0
    if (n == 'C'):
        a[index] = 0
        u[index] = 0
        c[index] = 1
        g[index] = 0
    if (n == 'G'):
        a[index] = 0
        u[index] = 0
        c[index] = 0
        g[index] = 1
    
    index += 1

In [9]:
print(a)
print(u)
print(c)
print(g)
print(s)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0}
{1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0}
{1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0}
{1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1}
CGUCUUCACUACAGCAUCGG


## X_ij

In [10]:
ls_tup = []
for i in range(1,len(s)):
    for j in range(i+1,len(s)+1):
        ls_tup.append((i,j))
        
x = m.addVars(ls_tup,name='X_ij',vtype=GRB.BINARY)
x

{(1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (

## d_ij Minimum Distance Constraint

In [11]:
ls_dtup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_dtup.append((i,j))

d = {}
d

{}

In [12]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        w = min(j-i,len(s)-j+i)
        d[i,j] = w

d

{(1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (1, 5): 4,
 (1, 6): 5,
 (1, 7): 6,
 (1, 8): 7,
 (1, 9): 8,
 (1, 10): 9,
 (1, 11): 10,
 (1, 12): 9,
 (1, 13): 8,
 (1, 14): 7,
 (1, 15): 6,
 (1, 16): 5,
 (1, 17): 4,
 (1, 18): 3,
 (1, 19): 2,
 (1, 20): 1,
 (2, 3): 1,
 (2, 4): 2,
 (2, 5): 3,
 (2, 6): 4,
 (2, 7): 5,
 (2, 8): 6,
 (2, 9): 7,
 (2, 10): 8,
 (2, 11): 9,
 (2, 12): 10,
 (2, 13): 9,
 (2, 14): 8,
 (2, 15): 7,
 (2, 16): 6,
 (2, 17): 5,
 (2, 18): 4,
 (2, 19): 3,
 (2, 20): 2,
 (3, 4): 1,
 (3, 5): 2,
 (3, 6): 3,
 (3, 7): 4,
 (3, 8): 5,
 (3, 9): 6,
 (3, 10): 7,
 (3, 11): 8,
 (3, 12): 9,
 (3, 13): 10,
 (3, 14): 9,
 (3, 15): 8,
 (3, 16): 7,
 (3, 17): 6,
 (3, 18): 5,
 (3, 19): 4,
 (3, 20): 3,
 (4, 5): 1,
 (4, 6): 2,
 (4, 7): 3,
 (4, 8): 4,
 (4, 9): 5,
 (4, 10): 6,
 (4, 11): 7,
 (4, 12): 8,
 (4, 13): 9,
 (4, 14): 10,
 (4, 15): 9,
 (4, 16): 8,
 (4, 17): 7,
 (4, 18): 6,
 (4, 19): 5,
 (4, 20): 4,
 (5, 6): 1,
 (5, 7): 2,
 (5, 8): 3,
 (5, 9): 4,
 (5, 10): 5,
 (5, 11): 6,
 (5, 12): 7,
 (5, 13): 8,
 (5, 14): 9,

In [13]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if d[i,j] < 3:
            x[i,j] = 0

x

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (2, 9): <gurobi.Var *Awaiting Model Update*>,
 (2, 10): <gurobi.Var *Awaiting Model Update*>,
 (2

## f_ij

In [14]:
ls_ftup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_ftup.append((i,j))
        


        
f = {}
f

{}

In [15]:
temp = ''
temp = 'x' + s
temp
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if (((temp[i] == 'C') & (temp[j] == 'G')) or ((temp[i] == 'G') & (temp[j] == 'C'))):
            f[i,j] = 3
        elif (((temp[i] == 'A') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'A'))):
            f[i,j] = 2
        elif (((temp[i] == 'G') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'G'))):
            f[i,j] = 0.1
        elif (((temp[i] == 'A') & (temp[j] == 'C')) or ((temp[i] == 'C') & (temp[j] == 'A'))):
            f[i,j] = 0.05
        else:
            f[i,j] = 0
            
f

{(1, 2): 3,
 (1, 3): 0,
 (1, 4): 0,
 (1, 5): 0,
 (1, 6): 0,
 (1, 7): 0,
 (1, 8): 0.05,
 (1, 9): 0,
 (1, 10): 0,
 (1, 11): 0.05,
 (1, 12): 0,
 (1, 13): 0.05,
 (1, 14): 3,
 (1, 15): 0,
 (1, 16): 0.05,
 (1, 17): 0,
 (1, 18): 0,
 (1, 19): 3,
 (1, 20): 3,
 (2, 3): 0.1,
 (2, 4): 3,
 (2, 5): 0.1,
 (2, 6): 0.1,
 (2, 7): 3,
 (2, 8): 0,
 (2, 9): 3,
 (2, 10): 0.1,
 (2, 11): 0,
 (2, 12): 3,
 (2, 13): 0,
 (2, 14): 0,
 (2, 15): 3,
 (2, 16): 0,
 (2, 17): 0.1,
 (2, 18): 3,
 (2, 19): 0,
 (2, 20): 0,
 (3, 4): 0,
 (3, 5): 0,
 (3, 6): 0,
 (3, 7): 0,
 (3, 8): 2,
 (3, 9): 0,
 (3, 10): 0,
 (3, 11): 2,
 (3, 12): 0,
 (3, 13): 2,
 (3, 14): 0.1,
 (3, 15): 0,
 (3, 16): 2,
 (3, 17): 0,
 (3, 18): 0,
 (3, 19): 0.1,
 (3, 20): 0.1,
 (4, 5): 0,
 (4, 6): 0,
 (4, 7): 0,
 (4, 8): 0.05,
 (4, 9): 0,
 (4, 10): 0,
 (4, 11): 0.05,
 (4, 12): 0,
 (4, 13): 0.05,
 (4, 14): 3,
 (4, 15): 0,
 (4, 16): 0.05,
 (4, 17): 0,
 (4, 18): 0,
 (4, 19): 3,
 (4, 20): 3,
 (5, 6): 0,
 (5, 7): 0,
 (5, 8): 2,
 (5, 9): 0,
 (5, 10): 0,
 (5, 11): 2,
 (

## Q_ij

In [16]:
ls_qtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_qtup.append((i,j))

q_ij = m.addVars(ls_qtup,name='Q_ij',vtype=GRB.BINARY)
q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (

## Q_nj

In [17]:
ls_njtup = []
for j in range(3,len(s)):
    ls_njtup.append((len(s),j))
    
    
q_nj = m.addVars(ls_njtup,name='Q_nj',vtype=GRB.BINARY)
q_nj

{(20, 3): <gurobi.Var *Awaiting Model Update*>,
 (20, 4): <gurobi.Var *Awaiting Model Update*>,
 (20, 5): <gurobi.Var *Awaiting Model Update*>,
 (20, 6): <gurobi.Var *Awaiting Model Update*>,
 (20, 7): <gurobi.Var *Awaiting Model Update*>,
 (20, 8): <gurobi.Var *Awaiting Model Update*>,
 (20, 9): <gurobi.Var *Awaiting Model Update*>,
 (20, 10): <gurobi.Var *Awaiting Model Update*>,
 (20, 11): <gurobi.Var *Awaiting Model Update*>,
 (20, 12): <gurobi.Var *Awaiting Model Update*>,
 (20, 13): <gurobi.Var *Awaiting Model Update*>,
 (20, 14): <gurobi.Var *Awaiting Model Update*>,
 (20, 15): <gurobi.Var *Awaiting Model Update*>,
 (20, 16): <gurobi.Var *Awaiting Model Update*>,
 (20, 17): <gurobi.Var *Awaiting Model Update*>,
 (20, 18): <gurobi.Var *Awaiting Model Update*>,
 (20, 19): <gurobi.Var *Awaiting Model Update*>}

## w_ij

In [18]:
ls_wtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_wtup.append((i,j))
        
w_ij = {}
w_ij

{}

In [19]:
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        if ((a[i] == 1) & (u[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 9
        elif ((a[i] == 1) & (u[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 21
        elif ((a[i] == 1) & (u[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 24
        elif ((a[i] == 1) & (u[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 13
            
        elif ((c[i] == 1) & (g[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 22
        elif ((c[i] == 1) & (g[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 33
        elif ((c[i] == 1) & (g[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 34
        elif ((c[i] == 1) & (g[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 24
            
        elif ((g[i] == 1) & (c[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 21
        elif ((g[i] == 1 )& (c[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 24
        elif ((g[i] == 1) & (c[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 33
        elif ((g[i] == 1) & (c[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 21
            
        elif ((u[i] == 1 )& (a[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 12
        elif ((u[i] == 1) & (a[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 23
        elif ((u[i] == 1) & (a[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 21
        elif ((u[i] == 1) & (a[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 16
        else:
            w_ij[i,j] = 1
            

w_ij

{(1, 4): 1,
 (1, 5): 1,
 (1, 6): 1,
 (1, 7): 1,
 (1, 8): 1,
 (1, 9): 1,
 (1, 10): 1,
 (1, 11): 1,
 (1, 12): 1,
 (1, 13): 1,
 (1, 14): 1,
 (1, 15): 1,
 (1, 16): 1,
 (1, 17): 1,
 (1, 18): 1,
 (1, 19): 34,
 (1, 20): 1,
 (2, 5): 1,
 (2, 6): 1,
 (2, 7): 1,
 (2, 8): 1,
 (2, 9): 21,
 (2, 10): 1,
 (2, 11): 1,
 (2, 12): 21,
 (2, 13): 1,
 (2, 14): 1,
 (2, 15): 1,
 (2, 16): 1,
 (2, 17): 1,
 (2, 18): 1,
 (2, 19): 1,
 (2, 20): 1,
 (3, 6): 1,
 (3, 7): 1,
 (3, 8): 1,
 (3, 9): 1,
 (3, 10): 1,
 (3, 11): 1,
 (3, 12): 1,
 (3, 13): 1,
 (3, 14): 1,
 (3, 15): 1,
 (3, 16): 1,
 (3, 17): 1,
 (3, 18): 1,
 (3, 19): 1,
 (3, 20): 1,
 (4, 7): 1,
 (4, 8): 1,
 (4, 9): 1,
 (4, 10): 1,
 (4, 11): 1,
 (4, 12): 1,
 (4, 13): 1,
 (4, 14): 24,
 (4, 15): 1,
 (4, 16): 1,
 (4, 17): 1,
 (4, 18): 1,
 (4, 19): 1,
 (4, 20): 1,
 (5, 8): 1,
 (5, 9): 1,
 (5, 10): 1,
 (5, 11): 1,
 (5, 12): 1,
 (5, 13): 1,
 (5, 14): 1,
 (5, 15): 1,
 (5, 16): 1,
 (5, 17): 1,
 (5, 18): 1,
 (5, 19): 1,
 (5, 20): 1,
 (6, 9): 1,
 (6, 10): 1,
 (6, 11): 1,
 (6

## w_nj

In [20]:
ls_wnjtup = []
for j in range(3,len(s)):
    ls_wnjtup.append((len(s),j))


w_nj = {}
w_nj

{}

In [21]:
for j in range(3,len(s)):
    if ((a[len(s)] == 1) & (u[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 9
    elif ((a[len(s)] == 1) & (u[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 21
    elif ((a[len(s)] == 1) & (u[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 24
    elif ((a[len(s)] == 1) & (u[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 13

    elif ((c[len(s)] == 1) & (g[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 22
    elif ((c[len(s)] == 1) & (g[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 33
    elif ((c[len(s)] == 1) & (g[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 34
    elif ((c[len(s)] == 1) & (g[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 24

    elif ((g[len(s)] == 1) & (c[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 21
    elif ((g[len(s)] == 1 )& (c[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 24
    elif ((g[len(s)] == 1) & (c[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 33
    elif ((g[len(s)] == 1) & (c[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 21

    elif ((u[len(s)] == 1 )& (a[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 12
    elif ((u[len(s)] == 1) & (a[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 23
    elif ((u[len(s)] == 1) & (a[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 21
    elif ((u[len(s)] == 1) & (a[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 16
    else:
        w_nj[len(s),j] = 1


w_nj

{(20, 3): 1,
 (20, 4): 1,
 (20, 5): 1,
 (20, 6): 1,
 (20, 7): 1,
 (20, 8): 1,
 (20, 9): 1,
 (20, 10): 1,
 (20, 11): 1,
 (20, 12): 1,
 (20, 13): 1,
 (20, 14): 1,
 (20, 15): 24,
 (20, 16): 1,
 (20, 17): 1,
 (20, 18): 1,
 (20, 19): 1}

## P_ij

In [22]:
ls_pijtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_pijtup.append((i,j))
        
p_ij = m.addVars(ls_pijtup,name='P_ij',vtype=GRB.BINARY)
p_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (

## P_nj

In [23]:
ls_pnjtup = []
for j in range(3,len(s)):
    ls_pnjtup.append((len(s),j))


p_nj = m.addVars(ls_pnjtup,name='P_nj',vtype=GRB.BINARY)
p_nj

{(20, 3): <gurobi.Var *Awaiting Model Update*>,
 (20, 4): <gurobi.Var *Awaiting Model Update*>,
 (20, 5): <gurobi.Var *Awaiting Model Update*>,
 (20, 6): <gurobi.Var *Awaiting Model Update*>,
 (20, 7): <gurobi.Var *Awaiting Model Update*>,
 (20, 8): <gurobi.Var *Awaiting Model Update*>,
 (20, 9): <gurobi.Var *Awaiting Model Update*>,
 (20, 10): <gurobi.Var *Awaiting Model Update*>,
 (20, 11): <gurobi.Var *Awaiting Model Update*>,
 (20, 12): <gurobi.Var *Awaiting Model Update*>,
 (20, 13): <gurobi.Var *Awaiting Model Update*>,
 (20, 14): <gurobi.Var *Awaiting Model Update*>,
 (20, 15): <gurobi.Var *Awaiting Model Update*>,
 (20, 16): <gurobi.Var *Awaiting Model Update*>,
 (20, 17): <gurobi.Var *Awaiting Model Update*>,
 (20, 18): <gurobi.Var *Awaiting Model Update*>,
 (20, 19): <gurobi.Var *Awaiting Model Update*>}

## L_ij

In [24]:
ls_lijtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_lijtup.append((i,j))
        
l_ij = m.addVars(ls_lijtup,name='L_ij',vtype=GRB.BINARY)
l_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (

## L_nj

In [25]:
ls_lnjtup = []
for j in range(3,len(s)):
    ls_lnjtup.append((len(s),j))


l_nj = m.addVars(ls_lnjtup,name='L_nj',vtype=GRB.BINARY)
l_nj

{(20, 3): <gurobi.Var *Awaiting Model Update*>,
 (20, 4): <gurobi.Var *Awaiting Model Update*>,
 (20, 5): <gurobi.Var *Awaiting Model Update*>,
 (20, 6): <gurobi.Var *Awaiting Model Update*>,
 (20, 7): <gurobi.Var *Awaiting Model Update*>,
 (20, 8): <gurobi.Var *Awaiting Model Update*>,
 (20, 9): <gurobi.Var *Awaiting Model Update*>,
 (20, 10): <gurobi.Var *Awaiting Model Update*>,
 (20, 11): <gurobi.Var *Awaiting Model Update*>,
 (20, 12): <gurobi.Var *Awaiting Model Update*>,
 (20, 13): <gurobi.Var *Awaiting Model Update*>,
 (20, 14): <gurobi.Var *Awaiting Model Update*>,
 (20, 15): <gurobi.Var *Awaiting Model Update*>,
 (20, 16): <gurobi.Var *Awaiting Model Update*>,
 (20, 17): <gurobi.Var *Awaiting Model Update*>,
 (20, 18): <gurobi.Var *Awaiting Model Update*>,
 (20, 19): <gurobi.Var *Awaiting Model Update*>}

## Additional Constraints for Q_ij and Q_nj variables

In [26]:
m.addConstrs(2*q_ij[i,j] <= x[i,j] + x[i+1,j-1] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model

In [27]:
m.addConstrs(2*q_nj[len(s),j] <= x[j,len(s)] + x[1,j-1] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>}

## Q_i,i+1

In [28]:
ls_ii1tup = []
for i in range(1,len(s)):
    ls_ii1tup.append((i,i+1))

m.addVars(ls_ii1tup,name='Q_ij',vtype=GRB.BINARY)
for i in range(1,len(s)):
    q_ij[i,i+1] = 0

q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (

## Q_i,i+2

In [29]:
ls_ii2tup = []
for i in range(1,len(s)-1):
    ls_ii2tup.append((i,i+2))

m.addVars(ls_ii2tup,name='Q_ij',vtype=GRB.BINARY)
for i in range(1,len(s)-1):
    q_ij[i,i+2] = 0

q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (

## Q_n,n-1

In [30]:
q_nj[len(s),len(s)-1] = 0

## Q_n,n-2

In [31]:
q_nj[len(s),len(s)-2] = 0

# Additional Constraints for P_ij, P_nj, L_ij, L_nj

In [32]:
m.addConstrs(q_ij[i,j] >= p_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model

In [33]:
m.addConstrs(q_ij[i,j] + (1-q_ij[i-1,j+1]) >= 2*p_ij[i,j] for i in range(2,len(s)-2) for j in range(i+3,len(s)))

{(2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (3, 7): <gurobi.Constr *Awaiting Model Update*>,
 (3, 8): <gurobi.Constr *Awaiting Model Update*>,
 (3, 9): <gurobi.Constr *Awaiting Model Update*>,
 (3, 10): <gurobi.Constr *Awaiting Model

In [34]:
m.addConstrs(q_ij[i,len(s)] + (1-q_ij[1,i-1]) >= 2*p_ij[i,len(s)] for i in range(3,len(s)-2))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>}

In [35]:
m.addConstrs(q_ij[1,j] + (1-q_nj[len(s),j+1]) >= 2 * p_ij[1,j] for j in range(4,len(s)-1))

{4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>}

In [36]:
m.addConstrs(q_nj[len(s),j] >= p_nj[len(s),j] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>}

In [37]:
m.addConstrs(q_nj[len(s),j] + (1-q_ij[j+1,len(s)-1]) >= 2*p_nj[len(s),j] for j in range(3,len(s)-2))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>}

In [38]:
m.addConstrs(q_ij[i,j] >= l_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model

In [39]:
m.addConstrs(q_ij[i,j] + (1-q_ij[i+1,j-1]) >= 2*l_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model

In [40]:
m.addConstrs(q_nj[len(s),j] + (1-q_ij[1,j-1]) >= 2*l_nj[len(s),j] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>}

In [41]:
m.addConstrs(p_ij[i,j] + l_ij[i,j] <= 1 for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model

In [42]:
m.addConstrs(p_nj[len(s),j] + l_nj[len(s),j] <= 1 for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>}

## Character in At most one pair

In [43]:
m.addConstrs(quicksum([x[i,k] for i in range(1,k)]) + quicksum([x[k,j] for j in list(range(k+1,len(s)+1))]) <= 1 for k in ONE_N)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>}

# At most 10 Crossing Pairs

In [44]:
ls_ijkl = []
for i in range(1,len(s)):
    for j in range(i+1,len(s)+1):
        for k in range(i+1,j):
            for l in range(j+1,len(s)+1):
                ls_ijkl.append((i,j,k,l))


c_ijkl = m.addVars(ls_ijkl,name='C_ijkl',vtype=GRB.BINARY)
c_ijkl

{(1, 3, 2, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 2, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 4, 2, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 4, 2, 6): <gurobi.Var *Awaitin

In [45]:
m.addConstr((quicksum(c_ijkl[i,j,k,l] for i in range(1,len(s)) for j in range(i+1,len(s)+1)
                      for k in range(i+1,j) for l in range(j+1,len(s)+1))) <= 10 )

<gurobi.Constr *Awaiting Model Update*>

In [46]:
m.addConstrs((x[i,j] + x[k,l] >= 2*c_ijkl[i,j,k,l]) for i in range(1,len(s)) for j in range(i+1,len(s)+1) 
             for k in range(i+1,j) for l in range(j+1,len(s)+1))

{(1, 3, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 2, 5): <gurobi.Constr *Awaiti

In [47]:
m.addConstrs(x[i,j] + x[k,l] - 1 <= c_ijkl[i,j,k,l] for i in range(1,len(s)) for j in range(i+1,len(s)+1) 
             for k in range(i+1,j) for l in range(j+1,len(s)+1))

{(1, 3, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 2, 5): <gurobi.Constr *Awaiti

# Objective

In [48]:
m.setObjective((quicksum(f[i,j]*x[i,j] for i in ONE_N for j in range(i+1,len(s)+1)) + 
               (quicksum((w_ij[i,j] * (q_ij[i,j] + p_ij[i,j] + l_ij[i,j])) for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))) + 
               (quicksum((w_nj[len(s),j] * (q_nj[len(s),j] + p_nj[len(s),j] + l_nj[len(s),j])) for j in range(3,len(s))))),GRB.MAXIMIZE)

In [49]:
# Optimize the model
m.optimize()
# Print the result
status_code = {1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD', 5: 'UNBOUNDED'}
status = m.status
print("The optimization status is {}".format(status_code[status]))
if status == 2:
# Retrieve variables value
    print("Optimal solution:")
    for v in m.getVars():
        if (v.x == 1):
            print("%s = %g" % (v.varName, v.x))
    print("Optimal objective value:\n{}".format(m.objVal))
    print("Runtime is:", m.Runtime)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 10709 rows, 5582 columns and 35951 nonzeros
Model fingerprint: 0xa89bc9e1
Variable types: 0 continuous, 5582 integer (5582 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e-02, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 0.3000000
Presolve removed 1250 rows and 605 columns
Presolve time: 0.11s
Presolved: 9459 rows, 4977 columns, 32967 nonzeros
Variable types: 0 continuous, 4977 integer (4977 binary)
Found heuristic solution: objective 13.2000000

Root relaxation: objective 1.490000e+02, 716 iterations, 0.05 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  149.00000    0   87   13.20000 

In [50]:
m.write("check.lp")